In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
sample_submission = pd.read_csv("../input/cat-in-the-dat-ii/sample_submission.csv")
test = pd.read_csv("../input/cat-in-the-dat-ii/test.csv",index_col = 'id')
train = pd.read_csv("../input/cat-in-the-dat-ii/train.csv",index_col = 'id')

In [ ]:
train.columns
#remove rows with missing target
train.dropna(axis=0, subset=['target'], inplace=True)
train_y=train.target
train.drop(['target'],axis=1, inplace=True)

In [ ]:
for col in train.columns.difference(["id"]):
    train_vals = set(train[col].dropna().unique())
    test_vals = set(test[col].dropna().unique())

    xor_cat_vals = train_vals ^ test_vals
    if xor_cat_vals:
        print(f"Replacing {len(xor_cat_vals)} values in {col}, {xor_cat_vals}")
        train.loc[train[col].isin(xor_cat_vals), col] = "xor"
        test.loc[test[col].isin(xor_cat_vals), col] = "xor"

In [ ]:
xor_cat_vals

In [ ]:
'''Variable Description'''
def description(df):
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values
    summary['PercMissing'] = df.isnull().sum().values / df.isnull().count().values
    summary['Uniques'] = df.nunique().values
    summary['First Value'] = df.iloc[0].values
    summary['Second Value'] = df.iloc[1].values
    summary['Third Value'] = df.iloc[2].values
    return summary
print('**Variable Description of  train Data:**')
description(train)

Fill in mode for all NaN values

In [ ]:
columns = train.columns
for c in columns:
    train[c].fillna(train[c].mode()[0],inplace=True)
    test[c].fillna(test[c].mode()[0],inplace=True)


In [ ]:
# import matplotlib.pyplot as plt
# %matplotlib inline
# piv = train.pivot_table(index='ord_0', values='target')
# piv.plot.bar()
# plt.show()

Use Label encoding

In [ ]:
columns_to_ord = ['bin_3','bin_4','ord_1', 'ord_2']
for col in columns_to_ord:
    print('{}'.format(col), train[col].unique(),'\n' )


In [ ]:
dict_bin_3 = {'T':1, 'F':0}
dict_bin_4 = {'Y': 1, 'N':0}
dict_ord_1 = {'Novice':1,'Contributor':2,'Expert':3,'Master':4,'Grandmaster':5, }
dict_ord_2 = {'Freezing':1,'Cold':2,'Warm':3,'Hot':4,'Boiling Hot':5,'Lava Hot':6}
# dict_ord_3 = {'a':1, 'b':2, 'c':3, 'd':4, 'e':5, 'f':6, 'g':7, 'h':8, 'i':9, 'j':10, 'k':11,
#               'l':12, 'm':13,'n':14, 'o':15}
# dict_ord_4 = {'A':1, 'B':2, 'C':3, 'D':4, 'E':5, 'F':6, 'G':7, 'H':8, 'I':9, 'J':10,'K':11,
#               'L':12, 'M':13,'N':14, 'O':15, 'P':16, 'Q':17, 'R':18, 'S':19, 'T':20, 'U':21,
#               'V':21, 'W':22, 'X':24, 'Y':25, 'Z':26}

In [ ]:
dict_list=[dict_bin_3, dict_bin_4,dict_ord_1,dict_ord_2]
cols = ['bin_3','bin_4','ord_1', 'ord_2']
for i in range(0,len(cols)):
    col = cols[i]
    d = dict_list[i]
    train[col]=train[col].replace(d)
    test[col]=test[col].replace(d)

In [ ]:
# map_ord5 = {key:value for value,key in enumerate(sorted(train.ord_5.unique()))} 
# train.ord_5 = train.ord_5.map(map_ord5)
# test.ord_5 = test.ord_5.map(map_ord5)

The remaining object columns are not ordinal so we will use target encoding.

In [ ]:
train.month=train.month.apply(lambda x: str(x))
train.day=train.day.apply(lambda x: str(x))
test.month=test.month.apply(lambda x: str(x))
test.day=test.day.apply(lambda x: str(x))

In [ ]:
cat_feat_to_encode=train.select_dtypes(include='object').columns

In [ ]:
cat_feat_to_encode

In [ ]:
import category_encoders as ce
smoothing = 0.2
oof = pd.DataFrame([])
from sklearn.model_selection import StratifiedKFold
for tr_idx, oof_idx in StratifiedKFold(n_splits=5, random_state=2020, shuffle=True).split(train, train_y):
    ce_target_encoder = ce.TargetEncoder(cols = cat_feat_to_encode, smoothing=smoothing)
    ce_target_encoder.fit(train.iloc[tr_idx, :], train_y.iloc[tr_idx])
    oof = oof.append(ce_target_encoder.transform(train.iloc[oof_idx, :]), ignore_index=False)
ce_target_encoder = ce.TargetEncoder(cols = cat_feat_to_encode, smoothing=smoothing)
ce_target_encoder.fit(train, train_y);  train = oof.sort_index(); test = ce_target_encoder.transform(test)

In [ ]:
train.head()

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
XGB = XGBClassifier(n_estimators = 500, learning_rate=0.01, objective='binary:logistic',
                         tree_method = 'hist', max_depth = 5) 

#lr = LogisticRegression(solver='liblinear', random_state=0)
scores_xgb = cross_val_score(XGB, train, train_y,
                               cv=5,
                               scoring='roc_auc')
print(scores_xgb.mean())

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
model_lr = LogisticRegression(solver='liblinear', random_state=0)

scores_lr = cross_val_score(model_lr, train, train_y,
                               cv=5,
                               scoring='roc_auc')
print(scores_lr.mean())

In [ ]:
from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import roc_auc_score

model_rf = RandomForestRegressor(n_estimators = 100, random_state=0)
# scores_rf = cross_val_score(model_rf, train, train_y,
#                                cv=5,
#                                scoring='roc_auc')
# print(scores_rf.mean())

In [ ]:
model_rf

In [ ]:
model_rf.fit(train, train_y)
pred_rf=model_rf.predict(test)

In [ ]:
model_lr.fit(train, train_y)
pred_lr=model_lr.predict(test)

In [ ]:
output=pd.DataFrame({'Id':test.index, 'Target':pred_rf})
output.to_csv('submission7.csv', index=False)

In [ ]:
train['target']=train_y
train.head()

In [ ]:
corr_matrix=train.corr()
import seaborn as sns
sns.heatmap(corr_matrix)